# Hello, Fargate

## VPC

For fargate, we build a vpc that provides the networking context used to tie together the compute
resources used for the Fargate cluster, as well as to security context around network access and flows,
load balancer presence, etc.

For this project, we want to run containers that do not have a public address tied to them, with their images
pulled from docker hub. The combination of these two factors means they need to run in private subnets that can
access the external internet via a NAT gateway.

We also want a load balancer that can distribute requests from the outside to container ports. This means
we need a load balancer associated with our public subnets.

In [ ]:
stack_prefix = 'fg1-'

In [ ]:
cf_bucket = 'ds-deploy-us-east-1'
template_name = 'vpc.yml'
vpc_stack = stack_prefix + 'vpc'

In [ ]:
%%bash -s "$template_name" "$cf_bucket"
echo $2
ls $1
aws s3 cp ./$1 s3://$2

In [ ]:
import boto3

client = boto3.client('cloudformation')

In [ ]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [ ]:
import os
region = os.environ['AWS_DEFAULT_REGION']
bucketRoot = form_s3_url_prefix(region) + '/' + cf_bucket
print bucketRoot

In [ ]:
response = client.create_stack(
    StackName=vpc_stack,
    TemplateURL=bucketRoot + '/' + template_name
)

print response

In [ ]:
# Wait for it...
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=vpc_stack
)

print 'stack created'

In [ ]:
# Extract outputs needed for the next stack component
response = client.describe_stacks(
    StackName=vpc_stack
)

outputs = response['Stacks'][0]['Outputs']
print outputs

private_subnet_1 = [o for o in outputs if o['OutputKey'] == 'PrivateSubnet1'][0]['OutputValue']
private_subnet_2 = [o for o in outputs if o['OutputKey'] == 'PrivateSubnet2'][0]['OutputValue']
public_subnet_1 = [o for o in outputs if o['OutputKey'] == 'PublicSubnet1'][0]['OutputValue']
public_subnet_2 = [o for o in outputs if o['OutputKey'] == 'PublicSubnet2'][0]['OutputValue']
lb_ingress_sg = [o for o in outputs if o['OutputKey'] == 'LBIngressSecurityGroup'][0]['OutputValue']
web_ingress_sg = [o for o in outputs if o['OutputKey'] == 'WebIngressSecurityGroup'][0]['OutputValue']
vpc_id = [o for o in outputs if o['OutputKey'] == 'VpcId'][0]['OutputValue']

print private_subnet_1
print private_subnet_2
print public_subnet_1
print public_subnet_2
print lb_ingress_sg
print web_ingress_sg
print vpc_id

## Load Balancer

In [ ]:
lb_template_name = 'lb.yml'
lb_stack = stack_prefix + 'lb'
load_balancer_name = stack_prefix + 'lb'

In [ ]:
%%bash -s "$lb_template_name" "$cf_bucket"
echo $2
ls $1
aws s3 cp ./$1 s3://$2

In [ ]:
response = client.create_stack(
    StackName=lb_stack,
    TemplateURL=bucketRoot + '/' + lb_template_name,
    Parameters=[
        {
            'ParameterKey': 'LoadBalancerName',
            'ParameterValue': load_balancer_name
        },
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpc_id
        },
        {
            'ParameterKey': 'PublicSubnet1',
            'ParameterValue': public_subnet_1
        },
        {
            'ParameterKey': 'PublicSubnet2',
            'ParameterValue': public_subnet_2
        },
        {
            'ParameterKey': 'LBSecurityGroup',
            'ParameterValue': web_ingress_sg
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
# Wait for it...
print 'wait for lb stack'
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=lb_stack
)

print 'stack created'

In [ ]:
# Extract outputs needed for the next stack component
response = client.describe_stacks(
    StackName=lb_stack
)

outputs = response['Stacks'][0]['Outputs']
print outputs

lb_target_arn = [o for o in outputs if o['OutputKey'] == 'TargetGroup'][0]['OutputValue']

print lb_target_arn


In [ ]:
fg_template_name = 'fargate.yml'
fg_stack = stack_prefix + 'fg'
load_balancer_name = stack_prefix + 'fg'

In [ ]:
%%bash -s "$fg_template_name" "$cf_bucket"
echo $2
ls $1
aws s3 cp ./$1 s3://$2

In [ ]:
response = client.create_stack(
    StackName=fg_stack,
    TemplateURL=bucketRoot + '/' + fg_template_name,
    Parameters=[
        {
            'ParameterKey': 'LoadBalancerTGArn',
            'ParameterValue': lb_target_arn
        },
        {
            'ParameterKey': 'PrivateSubnet1',
            'ParameterValue': private_subnet_1
        },
        {
            'ParameterKey': 'PrivateSubnet2',
            'ParameterValue': private_subnet_2
        },
        {
            'ParameterKey': 'LBIngressSecurityGroup',
            'ParameterValue': lb_ingress_sg
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
# Wait for it...
print 'wait for fargate stack'
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=fg_stack
)

print 'stack created'